In [ ]:
import os
try:
  import wget
except:
  !pip install wget
  import wget
import tarfile


out_dir = 'data/svhn'

train_32_32 = ('http://ufldl.stanford.edu/housenumbers/train_32x32.mat', 'train_32x32.mat')
test_32_32 = ('http://ufldl.stanford.edu/housenumbers/test_32x32.mat', 'test_32x32.mat')
extra_32_32 = ('http://ufldl.stanford.edu/housenumbers/extra_32x32.mat', 'extra_32x32.mat')

train_large = ('http://ufldl.stanford.edu/housenumbers/train.tar.gz', 'train.tar.gz')
test_large = ('http://ufldl.stanford.edu/housenumbers/test.tar.gz', 'test.tar.gz')
extra_large = ('http://ufldl.stanford.edu/housenumbers/extra.tar.gz', 'extra.tar.gz')

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
def download_data(url, filename, out_dir=out_dir):
    filename = os.path.join(out_dir, filename)

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    if not os.path.exists(filename):
        print(f"Downloading {filename}.")
        wget.download(url, filename)
        print()
    else:
        print(f"Skipping {filename} download (already exists)")


def extract_data(filename, out_dir=out_dir):
    filename = os.path.join(out_dir, filename)

    print(f"Extracting {filename}")
    with tarfile.open(filename) as tar:
        tar.extractall(out_dir)

download_data(*train_32_32)
download_data(*test_32_32)
download_data(*extra_32_32)

download_data(*train_large)
download_data(*test_large)
download_data(*extra_large)

extract_data(train_large[1])
extract_data(test_large[1])
extract_data(extra_large[1])


In [ ]:
# -*- coding: utf-8 -*-
from tensorflow import keras
import numpy as np
from PIL import Image
from pathlib import Path
from scipy import io
import h5py
import json

def to_one_hot(a, n):
    result = np.zeros(shape=(a.shape[0], n))
    result[np.arange(len(a)), a] = 1
    return result

def load_multiple_digits_data(dir='data/svhn', train=True, extra=False):

    def parse_digit_struct(file):
        print('file - ' + str(file))
        if Path(f"{file}.cache.json").exists() and os.stat(f"{file}.cache.json").st_size != 0:
            print('exist')
            with open(f"{file}.cache.json", "r") as f:
                images = json.load(f)
                print(f'Loaded cached image attrs from {file}.cache.json')
                return images

        f = h5py.File(file, 'r')
        print(f'Opened file {file}')

        names = f['digitStruct']['name']
        bbox = f['digitStruct']['bbox']

        def extract_name(i):
            return ''.join([chr(c[0]) for c in f[names[i][0]].value])

        def extract_attr(i, attr):
            attr = f[bbox[i].item()][attr]
            if len(attr) > 1:
                return [f[attr.value[j].item()].value[0][0] for j in range(len(attr))]
            else:
                return [attr.value[0][0]]

        images = {}
        print(f'Extracting image attrs from {file}: ', end='')
        for i in range(len(names)):
            name = extract_name(i)
            images[name] = {
                "label": extract_attr(i, 'label'),
                "top": extract_attr(i, 'top'),
                "left": extract_attr(i, 'left'),
                "height": extract_attr(i, 'height'),
                "width": extract_attr(i, 'width')
            }
            if i % 1000 == 0:
                print('.', end='', flush=True)
        print()

        with open(f"{file}.cache.json", 'w+') as f:
            json.dump(images, f)
        return images

    def process_images(dir):
        cache_file = Path(dir) / 'cache.npz'
        if cache_file.exists():
            f = np.load(cache_file)
            print(f'Loaded cached arrays for {dir}')
            return [v for k, v in f.items()]

        attrs = parse_digit_struct(Path(dir) / 'digitStruct.mat')

        x, y = [], []
        print(f'Processing images from {dir}: ', end='', flush=True)
        for i, name in enumerate(os.listdir(dir)):
            if name not in attrs:
                print('s', end='', flush=True)
                continue

            img = Image.open(Path(dir) / name)

            height = int(max(attrs[name]['height']))
            width = int(max(attrs[name]['width']))
            left = max(int(min(attrs[name]['left'])) - 0.5 * width, 0)
            top = max(int(min(attrs[name]['top'])) - 0.5 * height, 0)
            right = min(int(max(attrs[name]['left'])) + 1.5 * width, img.size[0])
            bottom = min(int(max(attrs[name]['top'])) + 1.5 * height, img.size[1])

            img = img.crop(box=(left, top, right, bottom))
            img = img.resize((96, 96))

            label = [d % 10 for d in attrs[name]['label']]
            if len(label) > 6:
                print('e', end='', flush=True)
                continue

            label += [10] * (6 - len(label))
            label = to_one_hot(np.array(label, dtype=np.int), 11)

            x.append(np.array(img))
            y.append(np.array(label))

            if i % 1000 == 0:
                print('.', end='', flush=True)
        print()

        x = np.array(x, dtype=np.uint8)
        y = np.array(y, dtype=np.uint8)
        np.savez(Path(dir) / "cache.npz", x, y)
        return x, y

    x_test, y_test = process_images(Path(dir) / 'test/')

    x_train, y_train = None, None
    if train:
        x_train, y_train = process_images(Path(dir) / 'train/')

    x_extra, y_extra = None, None
    if extra:
        x_extra, y_extra = process_images(Path(dir) / 'extra/')

    return (
        x_train, y_train,
        x_test, y_test,
        x_extra, y_extra
    )

    x_test, y_test = process_images(Path(dir) / 'test/')

    x_train, y_train = None, None
    if train:
        x_train, y_train = process_images(Path(dir) / 'train/')

    x_extra, y_extra = None, None
    if extra:
        x_extra, y_extra = process_images(Path(dir) / 'extra/')

    return (
        x_train, y_train,
        x_test, y_test,
        x_extra, y_extra
    )




In [ ]:
x_train, y_train, x_test, y_test, _, _ = load_multiple_digits_data()
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1)

def to_y(a, n):
    return [a[:,i,:] for i in range(n)]

if y_train is not None and y_val is not None and y_test is not None:
            y_train = to_y(y_train, 6)
            y_val = to_y(y_val, 6)
            y_test = to_y(y_test, 6)


In [ ]:
y_train[1].shape

In [ ]:
import tensorflow as tf

from tensorflow import keras

input = keras.layers.Input(shape=(96, 96, 3))

x = keras.layers.Conv2D(16, 5, activation='relu', padding='same')(input)
x = keras.layers.MaxPool2D(pool_size=(2, 2), padding = 'same')(x)
x = keras.layers.Conv2D(32, 5, activation='relu', padding='same')(x)
x = keras.layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
x = keras.layers.Conv2D(64, 5, activation='relu', padding='same')(x)
x = keras.layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(rate=0.1)(x)
x = keras.layers.Dense(100, activation='relu')(x)
x = keras.layers.Dropout(rate=0.1)(x)

out1 = keras.layers.Dense(11,  activation='linear')(x)
out2 = keras.layers.Dense(11,  activation='linear')(x)
out3 = keras.layers.Dense(11,  activation='linear')(x)
out4 = keras.layers.Dense(11,  activation='linear')(x)
out5 = keras.layers.Dense(11,  activation='linear')(x)
out6 = keras.layers.Dense(11,  activation='linear')(x)

model = keras.models.Model(
   inputs=[input],
   outputs=[out1,out2,out3,out4,out5,out6]
)
model.compile(
   optimizer=keras.optimizers.Adam(lr=0.001),
   loss='categorical_crossentropy',
   metrics=['categorical_accuracy']#,
   #loss_weights=[1, 1, 0.5, 0.3, 0.1, 0.05]
)
model.summary()

In [ ]:
model.fit(
                x_train,
                y_train,
                epochs=100,
                batch_size=32,
                verbose=2,
                validation_split=0.1,
                callbacks=[
                    keras.callbacks.EarlyStopping(
                        patience=10,
                        restore_best_weights=True
                    )
                ]
            )

In [ ]:
y_pred = model.predict(x_test)

total = np.array([True] * len(x_test))
for i, (y1, y2) in enumerate(zip(y_test, y_pred)):
  cur = np.argmax(y1, axis=1) == np.argmax(y2, axis=1)
  total = np.logical_and(total, cur)

  acc = np.mean(cur.astype(np.int))
  #history[f'test_out_{i}_acc'] = acc
  print(f'Accuracy of out_{i} = {acc:.5f}')

acc = np.mean(total.astype(np.int))
#history['test_acc'] = acc
print(f'Accuracy = {acc:.5f}')

In [ ]:
!ls data/svhn/

In [ ]:
_, _, x_test, y_test, x_train, y_train = load_multiple_digits_data(extra=True, train=False)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1)
if y_train is not None and y_val is not None and y_test is not None:
            y_train = to_y(y_train, 6)
            y_val = to_y(y_val, 6)
            y_test = to_y(y_test, 6)

In [ ]:
model.fit(
                x_train,
                y_train,
                epochs=100,
                batch_size=32,
                verbose=2,
                validation_split=0.1,
                callbacks=[
                    keras.callbacks.EarlyStopping(
                        patience=10,
                        restore_best_weights=True
                    )
                ]
            )

In [ ]:
y_pred = model.predict(x_test)

total = np.array([True] * len(x_test))
for i, (y1, y2) in enumerate(zip(y_test, y_pred)):
  cur = np.argmax(y1, axis=1) == np.argmax(y2, axis=1)
  total = np.logical_and(total, cur)

  acc = np.mean(cur.astype(np.int))
  #history[f'test_out_{i}_acc'] = acc
  print(f'Accuracy of out_{i} = {acc:.5f}')

acc = np.mean(total.astype(np.int))
#history['test_acc'] = acc
print(f'Accuracy = {acc:.5f}')

In [ ]:
input = keras.layers.Input(shape=(96, 96, 3))

mobile_net = keras.applications.mobilenet_v2.MobileNetV2(
  include_top=False,
  weights='imagenet',
  input_shape=(96, 96, 3),
  input_tensor=input,
  pooling='avg'
)

dropout = keras.layers.Dropout(rate=0.1)(mobile_net.output)

outputs = [
  keras.layers.Dense(11, activation='softmax', name=f'out_{i}')(dropout)
  for i in range(6)
]

model1 = keras.models.Model(
  inputs=[input],
  outputs=outputs
)
model1.compile(
  optimizer=keras.optimizers.Adam(lr=0.001),
  loss='categorical_crossentropy',
  metrics=['categorical_accuracy'],
  loss_weights=[1, 1, 0.5, 0.3, 0.1, 0.05]
)

In [ ]:
model.fit(
                x_train,
                y_train,
                epochs=100,
                batch_size=32,
                verbose=2,
                validation_split=0.1,
                callbacks=[
                    keras.callbacks.EarlyStopping(
                        patience=10,
                        restore_best_weights=True
                    )
                ]
            )

In [ ]:
y_pred = model1.predict(x_test)

total = np.array([True] * len(x_test))
for i, (y1, y2) in enumerate(zip(y_test, y_pred)):
  cur = np.argmax(y1, axis=1) == np.argmax(y2, axis=1)
  total = np.logical_and(total, cur)

  acc = np.mean(cur.astype(np.int))
  #history[f'test_out_{i}_acc'] = acc
  print(f'Accuracy of out_{i} = {acc:.5f}')

acc = np.mean(total.astype(np.int))
#history['test_acc'] = acc
print(f'Accuracy = {acc:.5f}')

In [ ]:
model.save_weights('models/svhn_multiple_mobile_net_extra/model')

In [ ]:
!ls models/svhn_multiple_mobile_net_extra/